# Quick test outside of Test macro

In [20]:
using GalacticDynamics
using Test
using PythonCall
using ODEInterfaceDiffEq
using DifferentialEquations


pyimport("sys")."path".append("")
pyimport("sys")."path".append("python")
accelerations_py = pyimport("accelerations")
au = pyimport("astropy.units")
gd = pyimport("gala.dynamics")
gp = pyimport("gala.potential")
gu = pyimport("gala.units")
gi = pyimport("gala.integrate")

Python: <module 'gala.integrate' from '/home/mmestre/.conda/envs/cosmos/lib/python3.11/site-packages/gala/integrate/__init__.py'>

In [14]:

t₁, t₂ = 0.0, 0.1
t_range = (t₁, t₂)
Δt = 0.01
x₀ = 10*[1,0,0]
v₀ = 100*[1,0,0]
m = 10^12*𝕦.m  # Msun
a = 20*𝕦.l
pot = NFW(m, a)
c = concentration(pot)
f(x) = log(1+x)-x/(1+x)
m_g = m/f(c)
usys = gu.UnitSystem(au.kpc, au.Gyr, au.Msun, au.radian, au.km/au.s, au.km/au.Gyr^2)
pot_Gala = gp.NFWPotential(Py(adimensional(m_g))*au.Msun, Py(adimensional(a))*au.kpc, units=usys)
w₀ = gd.PhaseSpacePosition(pos=Py(x₀)*au.kpc, vel=Py(v₀)*au.km/au.s)

Python: <PhaseSpacePosition cartesian, dim=3, shape=()>

In [17]:
orb_gala = pot_Gala.integrate_orbit(w₀, dt=Δt*au.Gyr, t1=t₁, t2=(t₂+Δt)*au.Gyr,
                                    Integrator=gi.DOPRI853Integrator)
@show orb_gala.x
sol = evolve(pot, x₀, v₀, t_range; options=ntSolverOptions(; saveat=Δt))
@show sol.x
@show 𝕤.ode

orb_gala.x = <py Quantity [10.        , 10.91844425, 11.6337007 , 12.15212205, 12.4781083 ,
           12.61434284, 12.56192863, 12.3204426 , 11.88791487, 11.26073038,
           10.43344028,  9.39845702] kpc>
sol.x = [10.0 10.895696546502945 11.588000254694084 12.083075368523923 12.385141191762845 12.496698660900227 12.418657763536984 12.150382676148439 11.689660289203934 11.032588216750872 10.173367496194926; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
𝕤.ode = Tsit5(; stage_limiter! = trivial_limiter!, step_limiter! = trivial_limiter!, thread = static(false),)


Tsit5(; stage_limiter! = trivial_limiter!, step_limiter! = trivial_limiter!, thread = static(false),)

In [31]:
sol = evolve(pot, x₀, v₀, t_range, Vern7(); options=ntSolverOptions(; saveat=Δt))
@show sol.x;

sol.x = [10.0 10.89569654712676 11.588000256102779 12.083075367940987 12.385141192395608 12.496698661487171 12.41865776477321 12.150382676478769 11.68966029373267 11.03258821127346 10.173367498333134; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]


In [32]:
sol = evolve(pot, x₀, v₀, t_range, ODEInterfaceDiffEq.dopri5(); options=ntSolverOptions(; saveat=Δt))
@show sol.x;

MethodError: MethodError: no method matching dopri5(::ODEInterfaceDiffEq.var"#12#18"{ODEProblem{StaticArraysCore.SVector{6, Float64}, Tuple{Float64, Float64}, false, NFW{Int64, Int64}, ODEFunction{false, SciMLBase.AutoSpecialize, typeof(ode), LinearAlgebra.UniformScaling{Bool}, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, typeof(SciMLBase.DEFAULT_OBSERVED), Nothing, Nothing, Nothing, Nothing}, Base.Pairs{Symbol, Union{}, Tuple{}, @NamedTuple{}}, SciMLBase.StandardODEProblem}, Tuple{Int64}}, ::Float64, ::Float64, ::StaticArraysCore.SVector{6, Float64}, ::ODEInterface.OptionsODE)

Closest candidates are:
  dopri5(::Any, ::Real, ::Real, !Matched::Vector, ::ODEInterface.AbstractOptionsODE)
   @ ODEInterface ~/.julia/packages/ODEInterface/RwRLn/src/Dopri5.jl:100
